In [21]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

In [22]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

# 데이터 불러오기

In [23]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv")
df_end = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_EndPrice_IPO.csv")

In [24]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

In [25]:
df_end_1 = df_end.drop(['종가 대비 3개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_3 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_6 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 3개월 수익률','NullCount'],axis =1)

# 공통 전처리

In [26]:
def process(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[-0.4, -0.2, 0.2, 0.4])
        df = df.drop(y_name,axis = 1)
        
    else :
        df['Cat'] = Cut(df[y_name],[-0.2, -0.1, 0.1, 0.2])
        df = df.drop(y_name,axis = 1)
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1)

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
    
    ## y_data 분포 확인 및 데이터프레임 생성
    df_1 = train_list[0]['Cat'].value_counts().reindex([1,2,3,4,5])
    df_2 = test_list[0]['Cat'].value_counts().reindex([1,2,3,4,5])
    df_y = pd.concat([df_1,df_2])
    
    
    for i in range(1,len(train_list)):
        a = train_list[i]['Cat'].value_counts().reindex([1,2,3,4,5])
        b = test_list[i]['Cat'].value_counts().reindex([1,2,3,4,5])
    
        new_data = pd.concat([a,b] ,axis =0)
        df_y = pd.concat([df_y,new_data],axis =1)
        
    df_y.columns = list(range(0,32))
    df_y.index = ['Train_1','Train_2','Train_3','Train_4','Train_5',\
              'Test_1','Test_2','Test_3','Test_4','Test_5']
    df_y.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
    
    return df_y

In [27]:
df_0_1 = process(df_0_1,'공모가 대비 1개월 수익률')

In [28]:
df_0_3 = process(df_0_3,'공모가 대비 3개월 수익률')

In [29]:
df_0_6 = process(df_0_6,'공모가 대비 6개월 수익률')

In [30]:
df_1_3 = process(df_1_3,'1_3_수익률')

In [31]:
df_3_6 = process(df_3_6,'3_6_수익률')

In [32]:
df_end_1 = process(df_end_1,'종가 대비 1개월 수익률')
df_end_3 = process(df_end_3,'종가 대비 3개월 수익률')
df_end_6 = process(df_end_6,'종가 대비 6개월 수익률')

In [33]:
df_0_1.to_csv("0_1_종속변수 분포.csv",encoding = 'euc-kr')
df_0_3.to_csv("0_3_종속변수 분포.csv",encoding = 'euc-kr')
df_0_6.to_csv("0_6_종속변수 분포.csv",encoding = 'euc-kr')
df_1_3.to_csv("1_3_종속변수 분포.csv",encoding = 'euc-kr')
df_3_6.to_csv("3_6_종속변수 분포.csv",encoding = 'euc-kr')

In [34]:
df_end_1

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
Train_1,13,10,9,10,10,8,7,6,3,3,...,9,9,9,11,9,10,11,11,11,10
Train_2,52,43,42,44,41,41,40,37,33,32,...,35,44,43,43,47,57,57,58,60,56
Train_3,85,86,84,91,84,81,79,83,81,87,...,155,152,157,149,151,136,135,134,144,144
Train_4,10,7,7,8,9,8,8,10,10,11,...,20,20,20,19,21,20,20,19,22,21
Train_5,8,8,8,7,5,5,5,13,12,13,...,24,17,18,17,16,10,10,10,11,9
Test_1,1,1,1,0,0,0,4,4,4,4,...,4,5,6,4,3,2,3,3,2,2
Test_2,8,8,10,7,7,9,8,6,7,6,...,20,19,21,21,17,9,15,13,10,8
Test_3,16,17,20,27,27,34,35,47,52,44,...,31,31,30,29,36,38,39,37,21,6
Test_4,4,3,3,3,2,3,4,6,6,6,...,5,6,6,6,7,6,7,7,3,1
Test_5,1,1,2,9,9,10,10,8,8,7,...,2,1,0,0,1,3,5,5,4,2


In [35]:
df_end_3

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
Train_1,23,19,18,17,13,11,13,10,8,8,...,17,18,18,20,19,23,23,23,22,25
Train_2,56,47,46,48,46,41,43,38,35,35,...,48,52,53,50,51,56,54,54,60,62
Train_3,64,62,61,66,61,62,53,57,55,63,...,122,129,131,124,127,115,113,112,121,111
Train_4,13,14,13,15,16,16,17,21,19,19,...,16,13,13,13,14,13,14,14,15,15
Train_5,12,12,12,14,13,13,13,23,22,21,...,40,30,32,32,33,26,29,29,30,27
Test_1,0,0,3,3,3,4,4,6,6,5,...,6,11,11,8,7,4,7,7,6,3
Test_2,8,8,9,6,6,11,14,15,17,13,...,17,19,18,17,16,17,26,25,18,10
Test_3,13,14,14,18,18,23,26,37,40,35,...,31,23,22,23,29,25,28,25,11,5
Test_4,4,3,4,7,6,6,6,2,3,3,...,4,2,4,4,4,6,4,4,2,0
Test_5,5,5,6,12,12,12,11,11,11,11,...,4,7,8,8,8,6,4,4,3,1


In [36]:
df_end_6

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
Train_1,38,30,28,26,21,16,17,16,14,15,...,30,30,32,31,34,38,38,37,37,39
Train_2,47,41,41,40,35,33,35,29,26,25,...,40,45,45,46,45,45,45,48,49,50
Train_3,56,56,55,61,62,64,57,58,56,63,...,110,115,116,109,110,107,106,103,116,107
Train_4,14,14,15,18,16,15,15,16,15,15,...,17,16,17,16,17,13,12,12,12,13
Train_5,13,13,11,15,15,15,15,30,28,28,...,46,36,37,37,38,30,32,32,34,31
Test_1,1,1,2,2,3,6,8,10,9,7,...,12,18,15,13,7,4,6,6,5,2
Test_2,3,3,5,5,5,7,8,11,13,11,...,13,11,13,13,14,18,25,22,17,9
Test_3,15,16,18,19,17,22,25,33,36,30,...,25,25,26,24,34,26,26,26,9,3
Test_4,6,5,4,2,2,3,3,5,6,6,...,3,2,1,2,1,4,5,4,4,1
Test_5,5,5,7,18,18,18,17,12,13,13,...,9,6,8,8,8,6,7,7,5,4


In [37]:
df_end_1.to_csv("end_1_종속변수 분포.csv",encoding = 'euc-kr')
df_end_3.to_csv("end_3_종속변수 분포.csv",encoding = 'euc-kr')
df_end_6.to_csv("end_6_종속변수 분포.csv",encoding = 'euc-kr')